<a href="https://colab.research.google.com/github/jinkyukim-me/Pytorch_Class_/blob/master/10_2_mnist_cnn_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.init
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
print(device)

cuda


In [4]:
torch.cuda.is_available()

True

In [0]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
#MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

In [0]:
#Dataloader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [0]:
#model
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        #Layer1
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #Layer2
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #FC - fully connected layer
        self.fc = nn.Linear(7*7*64, 10, bias = True) #0~9:10
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        

In [0]:
model = CNN().to(device)

In [10]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [0]:
#loss function
criterion = nn.CrossEntropyLoss().to(device)

In [0]:
#optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
#training
total_batch = len(data_loader)
print('Learning Started. It will takes some time.')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader: #X:input Y:label
        X = X.to(device) #input data
        Y = Y.to(device) #label data
        
        optimizer.zero_grad()
        hypothesis = model(X) #model에 input값을 넣으면 가설이 됨
        
        cost = criterion(hypothesis, Y) #가설과 실제 라벨 사이의 차이를 계산하는 loss
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch #cost값을 쌓아줌
        
    print('[Epoch:{}] cost = {}'.format(epoch +1, avg_cost))
print('Learning Finished!')

Learning Started. It will takes some time.
[Epoch:1] cost = 0.22356964647769928
[Epoch:2] cost = 0.062002651393413544
[Epoch:3] cost = 0.044692203402519226
[Epoch:4] cost = 0.03560512885451317
[Epoch:5] cost = 0.02901412732899189
[Epoch:6] cost = 0.02482593059539795
[Epoch:7] cost = 0.02084449492394924
[Epoch:8] cost = 0.018014147877693176
[Epoch:9] cost = 0.01514076441526413
[Epoch:10] cost = 0.012577877379953861
[Epoch:11] cost = 0.010160965844988823
[Epoch:12] cost = 0.009997643530368805
[Epoch:13] cost = 0.00806920975446701
[Epoch:14] cost = 0.006900227628648281
[Epoch:15] cost = 0.006571789737790823
Learning Finished!


In [14]:
#test
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:",accuracy.item())

Accuracy: 0.9892999529838562


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
